### This is a try to generate new molecules by learning latent space structure using VAE

#### I used as the decoder output a batch of idependent categorical distribution describing each 37 possible character in each position of the 50 chars long smile string.


#### I used a standard normal distribution as prior with a small weight(0.01) and without computing the exact KL divergence formula. Otherwise the result will not be satisfactory


#### I've correctly generated 24 new molecules using 10000 samples from the prior

In [2]:
import tensorflow as tf
#from tensorflow.keras.models import Sequential, Model
import tf_keras as tfk
import tensorflow_probability as tfp
import random
import numpy as np
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors

import tf_keras as tfk
tfkl = tfk.layers

from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

## Preparation of data

In [3]:
DATA_PATH = "c:\\users\\talibit\\Drug_discovery_data\\alzheimersdata.txt"
data = open(DATA_PATH, 'r').read().splitlines()
new = []
count =0
for molecule in data:
    count+=1
    if 30 < len(molecule) < 50 and count < 300000:
        if ('T' not in molecule) and ('V' not in molecule) and ('g' not in molecule) and ('L' not in molecule) and ('8' not in molecule):
            new.append(molecule)

max_seq_len = len(max(new, key=len)) 

l = [['$']+ list(i) for i in new]
#l = [['$']+ [i] for i in new]
data = pad_sequences(l, maxlen=max_seq_len+1, padding="post", value="£",dtype=object) 
print(data.shape)
# so now each smile string is 50 character long element
#data = np.expand_dims(data, -1)
type(data)

(847, 50)


numpy.ndarray

In [4]:
tokenizer = Tokenizer(num_words=None, char_level=True, lower=False) 

tokenizer.fit_on_texts(data) 
new_data = tokenizer.texts_to_sequences(data) 
new_data = np.array(new_data)
new_data = np.expand_dims(new_data,-1)
new_data.shape

(847, 50, 1)

In [5]:
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index)
print(f'number of words in word_index: {vocab_size}')
print(f'word_index: {word_index}')

number of words in word_index: 37
word_index: {'c': 1, '£': 2, 'C': 3, '(': 4, ')': 5, '1': 6, '2': 7, 'n': 8, 'O': 9, '3': 10, 'N': 11, '=': 12, '$': 13, '-': 14, '[': 15, ']': 16, '4': 17, 'H': 18, 'F': 19, '/': 20, 'o': 21, 'l': 22, 'S': 23, 'B': 24, 'r': 25, '@': 26, 's': 27, '#': 28, '\\': 29, '+': 30, '.': 31, '5': 32, 'I': 33, 'ï': 34, '»': 35, '¿': 36, 'P': 37}


In [6]:
seed = 1
np.random.seed(seed)
full_train, x_test = train_test_split(np.array(new_data), test_size=0.2, random_state=seed)

x_train, x_val = train_test_split(np.array(full_train), test_size=0.1, random_state=seed)

In [424]:
x_train.shape , x_val.shape, x_test.shape

((609, 50, 1), (68, 50, 1), (170, 50, 1))

In [7]:
## Now let's build a simple VAE that encode 50 characters logn smile string into 

latent_size = 10
Prior_distribution = tfd.MultivariateNormalDiag(loc=tf.zeros(latent_size, dtype=tf.float32),
                                        scale_diag=tf.ones(latent_size, dtype=tf.float32))


kl_regularizer= tfpl.KLDivergenceRegularizer(Prior_distribution,
                                                          weight = 0.01,
                                                          use_exact_kl = False, # using analytical formula does not lead to good results
                                                          test_points_fn = lambda q :q.sample(3),
                                                          test_points_reduce_axis = None) #None : all axis are averaged

In [8]:
encoder = tfk.Sequential([
        tfkl.LSTM(128, input_shape=(50,1), return_sequences=True),
        tfkl.Dropout(0.15),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(256, return_sequences=True),
        tfkl.Dropout(0.05),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(512, return_sequences=True),
        tfkl.Dropout(0.25),
        tfkl.Flatten(),
        tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
        tfpl.MultivariateNormalTriL(latent_size,
                                   activity_regularizer = kl_regularizer)


    ])
encoder.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 128)           66560     
                                                                 
 dropout (Dropout)           (None, 50, 128)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 25, 128)           0         
 D)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 25, 256)           394240    
                                                                 
 dropout_1 (Dropout)         (None, 25, 256)           0         
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 12, 256)           0         
 g1D)                                                 

In [9]:

decoder = tfk.Sequential([
    tfkl.Dense(10, activation = 'relu', input_shape =(latent_size,)),
    tfkl.Reshape((latent_size,1)),
    tfkl.LSTM(512, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(256, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(128, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.Flatten(),
    tfkl.Dense(50 * (vocab_size+1)),  # Output 50 * 37 logits
    tfkl.Reshape((50, vocab_size+1)),  # Reshape to (50, 37)
    tfpl.DistributionLambda(lambda t: tfd.Independent(
        tfd.Categorical(logits=t), reinterpreted_batch_ndims=1))

])
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 reshape (Reshape)           (None, 10, 1)             0         
                                                                 
 lstm_3 (LSTM)               (None, 10, 512)           1052672   
                                                                 
 dropout_3 (Dropout)         (None, 10, 512)           0         
                                                                 
 up_sampling1d (UpSampling1  (None, 20, 512)           0         
 D)                                                              
                                                                 
 lstm_4 (LSTM)               (None, 20, 256)           787456    
                                                      

In [10]:
vae = tfk.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs))

In [11]:
optimizer = tfk.optimizers.Adam(learning_rate=0.0005)
optimizer1 = tfk.optimizers.SGD()
vae.compile(optimizer=optimizer, loss=lambda x, pred : -tf.reduce_mean(pred.log_prob(x)))

In [12]:
#vae.fit(x = x_train, y = x_train,validation_data = (x_val,x_val), epochs = 1)
vae.fit(x = x_train, y = np.squeeze(x_train),validation_data = (x_val, np.squeeze(x_val)), epochs = 30)

Epoch 1/30

20/20 [==============================] - 21s 443ms/step - loss: 127.3292 - val_loss: 110.2631
Epoch 2/30
20/20 [==============================] - 5s 274ms/step - loss: 107.4685 - val_loss: 106.9287
Epoch 3/30
20/20 [==============================] - 5s 264ms/step - loss: 105.5072 - val_loss: 106.8990
Epoch 4/30
20/20 [==============================] - 5s 267ms/step - loss: 105.2314 - val_loss: 106.4367
Epoch 5/30
20/20 [==============================] - 5s 274ms/step - loss: 105.0442 - val_loss: 106.3398
Epoch 6/30
20/20 [==============================] - 5s 265ms/step - loss: 104.8943 - val_loss: 106.1647
Epoch 7/30
20/20 [==============================] - 6s 325ms/step - loss: 104.9059 - val_loss: 105.8790
Epoch 8/30
20/20 [==============================] - 6s 316ms/step - loss: 104.7728 - val_loss: 106.0722
Epoch 9/30
20/20 [==============================] - 6s 313ms/step - loss: 104.5940 - val_loss: 105.8938
Epoch 10/30
20/20 [==============================] - 6s 282ms/

In [13]:
# Evaluate the model on the test set

test_loss = vae.evaluate(x_test)
print("Test loss: {}".format(test_loss))

6/6 [==============================] - 3s 32ms/step - loss: 0.0501
Test loss: 0.05014514923095703


### Reconstruction of some examples

In [14]:
print(data.shape)

random_number = random.randint(0, 846)
print("the initial smile string is:",''.join(data[random_number,:].tolist()))

new_data = tokenizer.texts_to_sequences(data[random_number,:]) 
new_data = np.expand_dims(new_data,0)
new_data.shape
q = encoder(new_data)
q
p = decoder(q.mean())


sequence = p.sample().numpy()
tokenizer.sequences_to_texts(sequence)

(847, 50)
the initial smile string is: $COc1ccc(-c2nc3c(C(=O)Nc4cnccc4C)ccnc3[nH]2)cc1£££


['$ C C c 1 c 2 ) 1 ( c c 1 - o ] ] c ( c 1 - c ) c ) 4 C 1 3 2 + ] 2 n N ) C £ 2 c £ £ £ £ £ 1 1 £ £']

In [19]:
# Generation
n_samples = 10000
samples = Prior_distribution.sample(n_samples)
output_distribution = decoder(samples)
smiles = tokenizer.sequences_to_texts(output_distribution.sample().numpy())
#smiles

In [16]:
from rdkit import Chem
def checkSMILES(smiles):
	valid_smiles = []
	for smile in smiles:
		new = ''
		for char in smile:
			if char != '$' and char != '£':	
				new += char

		if Chem.MolFromSmiles(new, sanitize=False) is not None and len(new) > 15:
			valid_smiles.append(new)
			print(new)

	return valid_smiles

In [20]:
checkSMILES(smiles)

[17:47:55] SMILES Parse Error: syntax error while parsing: C C c 1 n ( C c C C 2 ( C N c ( ( ( c ( n n 3 c ] C n 2 = C C 2 O ) 3 ) O c c C 3  2  c    
[17:47:55] SMILES Parse Error: Failed parsing SMILES ' C C c 1 n ( C c C C 2 ( C N c ( ( ( c ( n n 3 c ] C n 2 = C C 2 O ) 3 ) O c c C 3  2  c    ' for input: ' C C c 1 n ( C c C C 2 ( C N c ( ( ( c ( n n 3 c ] C n 2 = C C 2 O ) 3 ) O c c C 3  2  c    '
[17:47:55] SMILES Parse Error: syntax error while parsing: C c c n c c c c c 2 n 1 H ] 2 3 4 c l c 1 ) ( n C c c 3 c ) O n = n O c C c c C n  C 2 2 O  O 
[17:47:55] SMILES Parse Error: Failed parsing SMILES ' C c c n c c c c c 2 n 1 H ] 2 3 4 c l c 1 ) ( n C c c 3 c ) O n = n O c C c c C n  C 2 2 O  O ' for input: ' C c c n c c c c c 2 n 1 H ] 2 3 4 c l c 1 ) ( n C c c 3 c ) O n = n O c C c c C n  C 2 2 O  O '
[17:47:55] SMILES Parse Error: syntax error while parsing: C c c 1 c c c - 2 c C 2 c n ( N 2 n [ c c l C c ) c ( N ) ) 2 N O c ( c 3 3 c  C c C  c c   
[17:47:55] SMILES Parse Error

O C O c 1 ) c 1 - N C C c C 2 c N c 3 N B C c  c ( c ( = n o = 4 c = o o  [ n =  C  1 n  . 2 =
O C S ( 1 c 2 c [ c C [ C c c 1 N c C = C ( ] ) 1 3 N ( ) ( ) c H c O ] ( - O ) 2 C )  O  )  O 
I C c c 1 c ( ( ( C 2 - = n ] ( 3 C l c ) 5 O c n c c ( = c N N 4 C c C ( 3 2 = c  n c C c    1
n 3 1 1 1 c ( c - ( 1 c n c r S c c 1 ) 2 ( N c N n [ ) ) ) C c 3 2 ] c [ 1 c 2  1 ) O 1   1 ] 
F C S c c c 1 1 C ( C C c c c n c ) ( ] ( c c ) 4 c n C = 2 C c 3 c s s c = c )  ) 1 ( ) )    
c O c ( n c c ] c s 2 2 C C C O c c o [ c ( C F 4 B n ] n 3 s ) F N ) l c O ( ) n 1 O  ] c c 2 ] 
o C C / 1 c c O c n 1 c [ = F N ( ( ( - / 4 ( / @ [ = ] c B C / O 1 c H ) c c »  N 2  2 c C F n I
n C O c c c c c c ( C 2 o O c C ] c n ( c 1 = l 2 ( c ) 2 N 2 4 ) 2 o - c F  c ) n 1 F 1    2 
B C O 1 n O c C ( ( c n N c 1 n N ) c N O H ( c N [ ( c n c ( ( H ) ) c o ) ) ] ( H N F    =  
C C O C c c ( C c n 2 ( 2 n O c c ) [ c C ( c 3 N n 2 C c ( 2 ) 2 1 = ) = 1 2 c 2 )   3     
N C c C 1 c c c c - C s c c 1 C ] - 3 c c =

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



['O C O c 1 ) c 1 - N C C c C 2 c N c 3 N B C c  c ( c ( = n o = 4 c = o o  [ n =  C  1 n  . 2 =',
 'O C S ( 1 c 2 c [ c C [ C c c 1 N c C = C ( ] ) 1 3 N ( ) ( ) c H c O ] ( - O ) 2 C )  O  )  O ',
 'I C c c 1 c ( ( ( C 2 - = n ] ( 3 C l c ) 5 O c n c c ( = c N N 4 C c C ( 3 2 = c  n c C c    1',
 'n 3 1 1 1 c ( c - ( 1 c n c r S c c 1 ) 2 ( N c N n [ ) ) ) C c 3 2 ] c [ 1 c 2  1 ) O 1   1 ] ',
 'F C S c c c 1 1 C ( C C c c c n c ) ( ] ( c c ) 4 c n C = 2 C c 3 c s s c = c )  ) 1 ( ) )    ',
 'c O c ( n c c ] c s 2 2 C C C O c c o [ c ( C F 4 B n ] n 3 s ) F N ) l c O ( ) n 1 O  ] c c 2 ] ',
 'o C C / 1 c c O c n 1 c [ = F N ( ( ( - / 4 ( / @ [ = ] c B C / O 1 c H ) c c »  N 2  2 c C F n I',
 'n C O c c c c c c ( C 2 o O c C ] c n ( c 1 = l 2 ( c ) 2 N 2 4 ) 2 o - c F  c ) n 1 F 1    2 ',
 'B C O 1 n O c C ( ( c n N c 1 n N ) c N O H ( c N [ ( c n c ( ( H ) ) c o ) ) ] ( H N F    =  ',
 'C C O C c c ( C c n 2 ( 2 n O c c ) [ c C ( c 3 N n 2 C c ( 2 ) 2 1 = ) = 1 2 c 2 )   3     ',
 'N

### We correctly generated 24 molecules  :)